
<a id="1"></a> <br>
## 2. Loading the data

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import gc
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn import metrics
import time, datetime
from sklearn import *
pd.set_option('display.max_columns', 500)

/Users/Enzo.Calogero/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
import feather
#test =feather.read_dataframe('tmp/microsoftv3_4_test')

#train =feather.read_dataframe('tmp/microsoft_train_prep_v1_2m')
#train =feather.read_dataframe('tmp/microsoft_train_prep_v2_2m')
                              #tmp/microsoft_train_prep_v1_full')
train =feather.read_dataframe('tmp/microsoft_train_optimized_for vaidations')
    #'tmp/trainwithdata')

In [3]:

#test =feather.read_dataframe('tmp/microsoftv3_train')#
#test =feather.read_dataframe('tmp/microsoft_test_prep_v2_2m')
    #'tmp/microsoft_test_prep_v1_full')  #'tmp/microsoftv3_train'
#train.info()
test =feather.read_dataframe('tmp/microsoft_test_prep_v2_fullb')

In [4]:
test.shape

(7853253, 90)

In [5]:
train.shape

(546372, 89)

In [6]:
target = train['HasDetections']
del train['HasDetections']

In [7]:
train['third']=train['third'].astype('category')
train['fourth']=train['fourth'].astype('category')
test['third']=test['third'].astype('category')
test['fourth']=test['fourth'].astype('category')

In [8]:
remove_variables=['Census_ActivationChannel',
 'Census_FlightRing',
 'Census_OSEdition',
 'Census_PowerPlatformRoleName',
 'Census_GenuineStateName',
 'Census_OSSkuName',
 'Census_OSBranch',
 'Census_OSVersion',
 'Census_InternalBatteryType',
 'ProductName',
 'Census_ChassisTypeName',
 'Census_OSArchitecture',
 'Census_OSInstallTypeName',
 'Census_OSWUAutoUpdateOptionsName']


In [9]:
print(len(train.columns))
train=train.drop(remove_variables, axis=1)
print(len(train.columns))

88
74


In [13]:
del train['test']
train.head(1)

,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_IsPortableOperatingSystem,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,enzoEngine,Enzoapp,Enzosign,first,second,third,fourth,fifth
0,1,1,1,0,7.0,0,NaN,0,0,1.0,1,82,-1,-1,6,6,0,0,0,0,1,0,0,1,1.0,0,NaN,0,-1,1.0,1.0,0,0,9,13,3,5.0,136,488386.0,1,487771.0,0,5,10,1024.0,1,4.294967e+09,17134,1,5,5,0,NaN,0.0,NaN,1,3360.0,0,NaN,0.0,0,0,0.0,1.0,4,14600.4,1317134.1,12630048.0,17134,1,0,0,17134.1


In [15]:
test.head()

,MachineIdentifier,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_InternalBatteryNumberOfCharges,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_IsPortableOperatingSystem,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,enzoEngine,Enzoapp,Enzosign,first,second,third,fourth,fifth
0,0000010489e3af074adeac69c53e555e,-1,-1,-1,0,7.0,0,NaN,53447.0,1.0,1.0,1,43,58552.0,18.0,53.0,42,0,0,0,15063,768,6,11,1,1.0,0,-1,NaN,108.0,-1,1.0,1.0,1,0,2689.0,30661.0,4.0,5.0,3063.0,-1,488386.0,1,123179.0,0,8192.0,15.5,1920.0,1080.0,8.0,15063,1387,37.0,158,0,NaN,0.0,NaN,807.0,8554.0,1,NaN,0.0,0,0,0.0,0.0,7.0,15400.5,1801810.5,12810501.0,15063,0,amd64fre,rs2_release,15063.0000
1,00000176ac758d54827acd545b6315a5,-1,4,-1,0,7.0,0,NaN,53447.0,1.0,1.0,1,68,71395.0,NaN,276.0,74,0,0,0,16299,768,2,2,1,1.0,0,-1,NaN,117.0,0,1.0,1.0,1,0,2206.0,242491.0,4.0,5.0,3410.0,-1,1907729.0,0,1882352.0,0,8192.0,15.5,1366.0,768.0,0.0,16299,611,7.0,30,0,NaN,0.0,NaN,554.0,33105.0,1,NaN,0.0,0,0,0.0,1.0,12.0,15400.4,1801809.2,12790301.0,16299,431,amd64fre,rs3_release_svc_escrow,16299.4310
2,0000019dcefc128c2d4387c1273dae1d,3,4,-1,0,7.0,0,NaN,49480.0,2.0,1.0,1,201,66202.0,NaN,267.0,-5,0,0,0,14393,768,1,16,1,1.0,0,-1,NaN,98.0,0,1.0,1.0,1,0,585.0,189533.0,2.0,5.0,2097.0,-1,29820.0,1,28678.0,0,4096.0,13.9,1280.0,720.0,0.0,14393,2189,35.0,148,0,NaN,0.0,NaN,556.0,63396.0,1,NaN,0.0,0,0,0.0,1.0,11.0,15300.6,1801809.2,12770230.0,14393,2189,amd64fre,rs1_release,14393.2189
3,0000055553dc51b1295785415f1a224d,-1,-1,-1,0,7.0,0,NaN,42160.0,2.0,1.0,1,29,120917.0,NaN,35.0,-85,0,0,0,16299,768,2,5,1,1.0,0,-1,NaN,117.0,0,1.0,1.0,1,0,2668.0,171228.0,2.0,5.0,1985.0,-1,476940.0,0,439345.0,0,4096.0,14.0,1366.0,768.0,0.0,16299,371,26.0,119,0,NaN,0.0,0.0,628.0,26320.0,1,0.0,0.0,0,0,0.0,0.0,10.0,15400.5,1801810.5,12810664.0,16299,15,amd64fre,rs3_release,16299.1500
4,00000574cefffeca83ec8adf9285b2bf,-1,4,-1,0,7.0,0,NaN,53447.0,1.0,1.0,1,171,124736.0,18.0,211.0,-74,0,0,0,16299,768,2,5,1,1.0,0,-1,NaN,117.0,0,1.0,1.0,1,0,585.0,189538.0,4.0,5.0,3394.0,-1,476940.0,0,461506.0,1,2048.0,15.5,1366.0,768.0,0.0,16299,371,29.0,125,0,NaN,0.0,NaN,556.0,63269.0,1,NaN,0.0,0,0,0.0,1.0,3.0,15400.4,1801809.2,12790236.0,16299,15,amd64fre,rs3_release,16299.1500


In [10]:
print(len(test.columns))
test=test.drop(remove_variables, axis=1)
print(len(test.columns))

90
76


In [11]:
...for sub sampling
print(train.shape)
train=train.sample(frac=0.2, replace=False)
print(train.shape)

SyntaxError: invalid syntax (<ipython-input-11-da75977adadb>, line 1)

<a id="3"></a> <br>
## 3. Training the model

We set the max number of iteration over folds:

In [17]:

train.shape

(546372, 73)

In [18]:

param = {'objective':'binary',
          "boosting": "gbdt",
          'learning_rate': 0.02,
          'max_depth': -1, 
          "feature_fraction": 0.8,
          "bagging_freq": 1,
          "bagging_fraction": 0.8 ,
          "bagging_seed": 11,
          "metric": 'auc',
           "lambda_l1": 0.1,
          'num_leaves': 60,
          'min_data_in_leaf': 60,
          "verbosity": -1,
          "random_state": 3}
auc=list()
for i in [1]:
    x1, x2, y1, y2 = model_selection.train_test_split(train, target, test_size=0.2)#, random_state=25)
    num_round = 10000
    model = lgb.train(param,
                      lgb.Dataset(x1, y1), #categorical_feature = categorical_features),
                      num_round,
                      lgb.Dataset(x2, y2), #categorical_feature = categorical_features),
                      verbose_eval=100,
                      early_stopping_rounds=200)
    auc.append(model.best_score['valid_0']['auc'])
print(auc)    
#best 120,40 .7365
# 120,60 .7354
# 120,100 .7349
# 100, 100 .7343
# 100,60 .7340
# 120,30 .7337

/Users/Enzo.Calogero/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/lightgbm/basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.681055
[200]	valid_0's auc: 0.688565
[300]	valid_0's auc: 0.69292
[400]	valid_0's auc: 0.695459
[500]	valid_0's auc: 0.696975
[600]	valid_0's auc: 0.697856
[700]	valid_0's auc: 0.698507
[800]	valid_0's auc: 0.698977
[900]	valid_0's auc: 0.69942
[1000]	valid_0's auc: 0.699708
[1100]	valid_0's auc: 0.69989
[1200]	valid_0's auc: 0.700102
[1300]	valid_0's auc: 0.700357
[1400]	valid_0's auc: 0.700554
[1500]	valid_0's auc: 0.700719
[1600]	valid_0's auc: 0.700925
[1700]	valid_0's auc: 0.70102
[1800]	valid_0's auc: 0.701167
[1900]	valid_0's auc: 0.701276
[2000]	valid_0's auc: 0.701423
[2100]	valid_0's auc: 0.701529
[2200]	valid_0's auc: 0.701587
[2300]	valid_0's auc: 0.701628
[2400]	valid_0's auc: 0.701708
[2500]	valid_0's auc: 0.701768
[2600]	valid_0's auc: 0.701879
[2700]	valid_0's auc: 0.701895
[2800]	valid_0's auc: 0.702039
[2900]	valid_0's auc: 0.702055
[3000]	valid_0's auc: 0.702081
[3100]	valid_0's auc

<a id="5"></a> <br>
## 5. Submission

In [19]:
features=train.columns

In [20]:
train.shape

(546372, 73)

In [21]:
predictions = model.predict(test[features], num_iteration=model.best_iteration)

In [22]:
sub_df = pd.DataFrame({"MachineIdentifier": test["MachineIdentifier"].values})
sub_df["HasDetections"] = predictions
sub_df[:10]

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0.380888
1,00000176ac758d54827acd545b6315a5,0.251450
2,0000019dcefc128c2d4387c1273dae1d,0.330110
3,0000055553dc51b1295785415f1a224d,0.381231
4,00000574cefffeca83ec8adf9285b2bf,0.373134
5,000007ffedd31948f08e6c16da31f6d1,0.265796
6,000008f31610018d898e5f315cdf1bd1,0.235970
7,00000a3c447250626dbcc628c9cbc460,0.208348
8,00000b6bf217ec9aef0f68d5c6705897,0.437719
9,00000b8d3776b13e93ad83676a28e4aa,0.167909


In [23]:
sub_df.to_csv("submitVersion2_15.csv", index=False)

# New heading